In [2]:
import random
import math
from typing import List

In [3]:
def convert_to_num(char: str) -> int:
    if char.isdigit():
        return int(char)
    elif char.isalpha():
        return ord(char.lower()) - 87
    else:
        return 36

def convert_to_char(num: int) -> str:
    if 0 <= num <= 9:
        return str(num)
    elif 10 <= num <= 35:
        return chr(num + 87)
    else:
        return ' '

def encode_message(msg: str) -> int:
    # Convert any extra characters to spaces
    msg = ''.join([c if c.isalnum() else ' ' for c in msg])

    # Pad the message with spaces to make sure it's divisible by 5
    msg += ' ' * (5 - len(msg) % 5) * (len(msg) % 5 != 0)

    # Split the message into groups of 5 characters
    groups = [msg[i:i+5] for i in range(0, len(msg), 5)]

    # Convert each group into a number using the specified formula
    nums = []
    for group in groups:
        num = 0
        for i, c in enumerate(group):
            num += convert_to_num(c) * (37 ** (4-i))
        nums.append(num)
    # Combine the numbers into a single integer by concatenating digits
    result = int(''.join([str(n) for n in nums]))
    return result

def decode_message(num: int) -> str:
    num_str = str(num).zfill((len(str(num)) + 7) // 8 * 8)

    # split into 8-character chunks and convert back to integers
    groups = [int(num_str[i:i+8]) for i in range(0, len(num_str), 8)]

    # Convert each group back into a string of 5 characters using mod and div
    msg = ''
    for group in groups:
        group_str = ''
        for i in range(5):
            c = convert_to_char(group // (37 ** (4-i))) 
            group_str += c
            group %= 37 ** (4-i)
        msg += group_str.rstrip()

    return msg

In [4]:
def extended_euclidean_algorithm(a: int, b: int) -> tuple:
    if b == 0:
        return (1, 0, a)
    else:
        x, y, gcd = extended_euclidean_algorithm(b, a % b)
        return (y, x - (a // b) * y, gcd)

In [5]:
def generate_keypair(p: int, q: int) -> tuple:
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Choose e such that e and phi(n) are coprime.
    # In practice, e is usually a small prime number, such as 65537.
    e = 7
    while math.gcd(e, phi) != 1:
        e = random.randint(2, phi - 1)
    
    # Use the extended Euclidean algorithm to compute the private key.
    # d is the multiplicative inverse of e modulo phi(n).
    # That is, d * e = 1 (mod phi(n)).
    x, y, gcd = extended_euclidean_algorithm(e, phi)
    d = x % phi
    
    # Return the public and private keys as a tuple.
    public_key = (e, n)
    private_key = (d, n)
    return (public_key, private_key)

In [6]:
def encrypt(public_key: tuple, message: int) -> int:
    e, n = public_key
    return pow(message, e, n)

def decrypt(private_key: tuple, encrypted_message: int) -> int:
    d, n = private_key
    return pow(encrypted_message, d, n)

In [7]:
import sympy
def generate_prime(bit_length):
    while True:
        p = sympy.randprime(2**(bit_length-1), 2**bit_length - 1)
        return p

def generate_two_distinct_primes(bit_length = 1024):
    while True:
        p = generate_prime(bit_length)
        q = generate_prime(bit_length)
        if p != q:
            return p, q
        

In [8]:
def Encrypt(message: str, public_key: tuple) -> int:
    encoded_message = encode_message(message)
    encrypted_message = encrypt(public_key, encoded_message)
    return encrypted_message

def Decrypt(encrypted_message: int, private_key: tuple) -> str:
    decoded_message = decrypt(private_key, encrypted_message)
    message = decode_message(decoded_message)
    return message

In [9]:
p , q = generate_two_distinct_primes()
print(p*q)
public_key, private_key = generate_keypair(p, q)

msg = "hi"
print("Original message:", msg)
encoded = encode_message(msg)

print(f"The encoded message is {encoded}")

encrypted_message = encrypt(public_key, encoded)
print("Encrypted message:", encrypted_message)

decrypted_message = decrypt(private_key, encrypted_message)
print("Decrypted message:", decrypted_message)
decoded = decode_message(decrypted_message)
print(f"The decoded message is '{decoded}'")

21219639835464654470642762616493764749672709575511311078098424654085680974459523727944411968666002212481859366800527090050929973288318218312757803353981287628767696463731609414509082194245218826433597195309822935685973775531572883796087866743932250543794407491388799934583305761526824371818970923955986633288921845719334705760184765243039172087723654988730607013860383803792149119465871328584322162087653614669011574107850284149271969999334198658930730561881330264986494243842563860787216470444326312224813640844795371951720489228299320448624039012881657965128079976181256761604726168412292850734139503844752420402837
Original message: hi
The encoded message is 32823143
Encrypted message: 41045084480298214856771364629621539625048913192864407
Decrypted message: 32823143
The decoded message is 'hi'


In [10]:
import numpy as np
import time
import matplotlib.pyplot as plt

def factorize(n):
   if (n % 2) == 0:
      return [2] + factorize(n//2)
   
   integer = 3
   while integer <= (n**0.5):     
      if n % integer == 0:      
         return [integer] + factorize(n // integer)
      else:
         integer += 2                        # Since all primes are odd.
   return [n]


def mathematical_attack(PU):
   factors = factorize(PU)
   return "NOT RSA" if len(factors) > 2  else factors

# n_sizes = np.arange(14, 32)
# times = []
# for size in n_sizes:
#     p,q = generate_two_distinct_primes(size)
#     start = time.time()
#     mathematical_attack(p*q)  
#     end = time.time()
#     times.append(end-start)

# plt.plot(n_sizes, times, 'bo-')
# plt.xlabel('RSA key size (bits)')
# plt.ylabel('Execution time (seconds)')
# plt.title('RSA key generation performance')
# plt.show()

In [11]:
def generate_keypair_given_q(p: int, q: int, e: int) -> tuple:
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Choose e such that e and phi(n) are coprime.
    # In practice, e is usually a small prime number, such as 65537.
    e = 7
    while math.gcd(e, phi) != 1:
        e = random.randint(2, phi - 1)
    
    # Use the extended Euclidean algorithm to compute the private key.
    # d is the multiplicative inverse of e modulo phi(n).
    # That is, d * e = 1 (mod phi(n)).
    x, y, gcd = extended_euclidean_algorithm(e, phi)
    d = x % phi
    
    # Return the public and private keys as a tuple.
    public_key = (e, n)
    private_key = (d, n)
    return (public_key, private_key)


message = "hello"
p0,q0 = generate_two_distinct_primes(20)
public_key_original , private_key_original = generate_keypair(p0,q0)
# print(public_key_original , private_key_original)
encrypted = Encrypt(message, public_key_original)
print("message :" , message )
print("cipher :" , encrypted )

e , n = public_key_original
p , q = mathematical_attack(n)
public_key , private_key = generate_keypair_given_q(p,q,e)
hopecracked = Decrypt(encrypted,private_key)
print("attacked :" , hopecracked )

message : hello
cipher : 33413556418
attacked : hello
